In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import keras
import json
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import itertools
import numpy as np
import os, shutil, io, glob
from datetime import datetime
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from collections import Counter
from keras.models import model_from_json

In [ ]:
test_images = glob.glob('../input/cassava-leaf-disease-classification/test_images/*.jpg')
df_test = pd.DataFrame(test_images, columns = ['path'])

IMG_SIZE = 224
SIZE = (IMG_SIZE,IMG_SIZE)

test_datagen = ImageDataGenerator()

test_gen= test_datagen.flow_from_dataframe(dataframe=df_test,
                                                x_col="path",
                                                y_col=None,
                                                batch_size=32,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=SIZE) ## (height, width)


In [ ]:
#json_file = open('../input/resnet/resnet.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#model = model_from_json(loaded_model_json)
#model.load_weights("../input/resnet/resnet.h5")
model = tf.keras.models.load_model('../input/stacked-model-v2-low-epsilon/stacked_models_v2')

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
pred_test = model.predict(test_gen,  verbose = True)
pred_test_labels = np.argmax(pred_test, axis = -1)

final_submission = df_test
final_submission['image_id'] = final_submission.path.str.split('/').str[-1]
final_submission['label'] = pred_test_labels

final_csv = final_submission[['image_id', 'label']]
final_csv.head()

final_csv.to_csv('submission.csv', index=False)